In [1]:
from datetime import datetime, timedelta
import cx_Oracle
import warnings
warnings.filterwarnings("ignore")

cx_Oracle.init_oracle_client(lib_dir=r"/Users/kaka/Downloads/instantclient_18_1")

import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import Ridge
import calendar
from datetime import datetime

# 获取当前日期
current_date = datetime.now()
# 获取当前月份的最后一天
last_day = calendar.monthrange(current_date.year, current_date.month)[1]
# 生成最后一天的日期字符串形式
last_day_str = f"{current_date.year}-{current_date.month:02d}-{last_day:02d}"

from datetime import datetime, timedelta


class SkuDemodsPred(object):

    def mysql_info(self):
        """数据上传"""
        username = 'nppbuf'
        password = 'Svwnppbuf321'
        hostname = '10.122.6.59'
        port = 1521
        service_name = 'b2bt'
        dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
        # 连接 Oracle 数据库
        connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
        return connection

In [2]:

def get_data_from_oracle(connection):

    sql_query = """  
         select *
      from nppbuf.T_DD_FORECAST_DETAIL_FJHIS 
      where HOSTPARTID  = 'KIT998648' and historybegdate>20220101
    """
    # 执行 SQL 查询，并按批次抽取数据
    chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值
    query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
    # 遍历查询结果的每个批次，并将其合并为一个 DataFrame
    data = pd.DataFrame()
    for chunk in query_chunks:
        data = data.append(chunk)
    # 关闭数据库连接
    connection.close()
    # 输出合并后的 DataFrame
    return data


In [3]:
ods = SkuDemodsPred()
connection = ods.mysql_info()
data = get_data_from_oracle(connection)
data.head()

,HOSTPARTID,HOSTLOCID,HISTORYBEGDATE,DSHOSTID,HISTORYAMOUNT,ABC_HITS_NETWORK,ABC_COST,ABC_DEMAND,AA,BB,CC
0,KIT998648,1000-1,20230530,MANUAL,10,A,D,ZERO,9,Y,Y
1,KIT998648,1000-1,20230530,MANUAL,10,A,D,ZERO,9,Y,Y
2,KIT998648,1000-1,20230530,MANUAL,50,A,D,ZERO,9,Y,Y
3,KIT998648,1000-1,20230530,MANUAL,50,A,D,ZERO,9,Y,Y
4,KIT998648,1000-1,20230530,MANUAL,15,A,D,ZERO,9,Y,Y


In [4]:

def last_days_of_month_and_six_months_ago(date):
    # 获取当前月份的最后一天
    last_day_current_month = date.replace(day=1, month=date.month + 1) 

    # 计算6个月前的日期
    six_months_ago = date - timedelta(days=180)

    # 获取6个月前月份的最后一天
    last_day_six_months_ago = six_months_ago.replace(day=1, month=six_months_ago.month + 1) 

    return last_day_current_month.strftime("%Y-%m-%d"), last_day_six_months_ago.strftime("%Y-%m-%d")

current_date = datetime.now()
now_time_future, end_time_train = \
            last_days_of_month_and_six_months_ago(current_date)
    

def data_preprocessing(data):
    data.columns = ['零件号', '仓库代码', '年月', '需求流', '需求数量', 'ABC需求频次分类',
                      'ABC价格分类', 'ABC需求数量分类', '零件号第四位','组装包','是否预测']
    data=data[data['是否预测']=='Y']
    data['日期'] = pd.to_datetime(data['年月'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    data = data.groupby(['零件号', '仓库代码', '需求流', '日期'])['需求数量'].sum().reset_index()
    data['日期'] = pd.to_datetime(data['日期'])
    """补零"""
    date_range = pd.date_range(start='2022-01-01', end=now_time_future, freq='D')
    # 创建空的DataFrame，准备存储填充后的结果
    filled_df = pd.DataFrame()
    # 针对每个 SKU 进行填充操作
    for sku, group in data.groupby(['零件号', '仓库代码', '需求流']):
        sku_group = group.set_index('日期').reindex(date_range, fill_value=0).reset_index()
        sku_group['零件号'] = sku[0]
        sku_group['仓库代码'] = sku[1]
        sku_group['需求流'] = sku[2]
        filled_df = filled_df.append(sku_group, ignore_index=True)
    return filled_df
data_pre = data_preprocessing(data)

In [5]:
def feature_processing(data,type):
    if type =='train':
        end_time = end_time_train
    else:
        end_time = now_time_future
    """生成列维度时间序列"""
    
    data['合并列'] = data['零件号'] + '_' + data['仓库代码'] + '_' + data['需求流']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库代码', '需求流'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index < end_time].copy()
    data_test = new_df[new_df.index >= end_time].copy()
    return data_train,data_test

In [6]:
data_train,data_test = feature_processing(data_pre,'train')

sku_list = data_train.columns

# 创建空的字典，用于存储每个时间序列的预测模型
forecasters = {}
# 循环遍历每批次SKU，分别训练预测模型
for i in range(0, len(sku_list), 200):
    batch_skus = sku_list[i:i + 200]
    for sku in batch_skus:
        forecaster = ForecasterAutoreg(  
        regressor=Ridge(random_state=123),  
        lags=30,  
                )
        # 拟合模型
        forecaster.fit(y=data_train[sku])
        forecasters[sku] = forecaster
    # print(f"Finished training batch {i // self.batch_size + 1}/{len(sku_list) // self.batch_size + 1}")
# 进行未来预测
predictions = pd.DataFrame()
for sku, forecaster in forecasters.items():
    forecast = forecaster.predict(steps=180)
    predictions[sku] = forecast
predictions[predictions < 0.1] = 0

In [7]:
def demods_groby_month(data,type):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info.name = type
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码', '需求流']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns = ['year', 'month', 'hostpartid', 'pred_values', 'hostlocid', 'dshostid']
    data_prs_info['pred_values'] = data_prs_info['pred_values'].astype(float)
    return data_prs_info[[ 'hostpartid', 'hostlocid', 'dshostid','year', 'month','pred_values']]

In [8]:
his_info = demods_groby_month(data_train,'pred_values')
thr_std = his_info.groupby(['hostpartid','hostlocid','dshostid'])['pred_values'].std().reset_index()
thr_std['pred_values'] = [x*3 for x in thr_std['pred_values']]
thr_std

,hostpartid,hostlocid,dshostid,pred_values
0,KIT998648,1000-1,DEALER,0.000000
1,KIT998648,1000-1,MANUAL,126410.886021


In [9]:
# data_prs = predictions.reset_index()
# data_prs = data_prs.rename(columns={'index': 'date'})
# data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
# data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
# data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
# data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
# data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
# data_prs_info.name = type
# data_prs_info = data_prs_info.reset_index()
# data_prs_info[['零件号', '仓库代码', '需求流']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
# data_prs_info.columns = ['year', 'month', 'hostpartid', 'pred_values', 'hostlocid', 'dshostid']
# data_prs_info['pred_values'] = data_prs_info['pred_values'].astype(float)
# data_prs_info

In [10]:
pred_info = demods_groby_month(predictions,'pred_values')
pred_info['pred_values'] = pred_info['pred_values'].round(2)
full_pred_info = pd.merge(pred_info,thr_std,on =['hostpartid','hostlocid','dshostid'],how ='left')
full_pred_info['pred_values'] = full_pred_info.apply(lambda row: row['pred_values_y'] if row['pred_values_x'] > row['pred_values_y'] else row['pred_values_x'], axis=1)
full_pred_info = full_pred_info.drop(['pred_values_x','pred_values_y'],axis =1 )
output = full_pred_info.append(his_info)

In [11]:
output.sort_values(by=['hostpartid','hostlocid','dshostid','year','month'],inplace =True)

In [12]:
output

,hostpartid,hostlocid,dshostid,year,month,pred_values
0,KIT998648,1000-1,DEALER,2022,1,0.00
2,KIT998648,1000-1,DEALER,2022,2,0.00
4,KIT998648,1000-1,DEALER,2022,3,0.00
6,KIT998648,1000-1,DEALER,2022,4,0.00
8,KIT998648,1000-1,DEALER,2022,5,0.00
10,KIT998648,1000-1,DEALER,2022,6,0.00
12,KIT998648,1000-1,DEALER,2022,7,0.00
14,KIT998648,1000-1,DEALER,2022,8,0.00
16,KIT998648,1000-1,DEALER,2022,9,0.00
18,KIT998648,1000-1,DEALER,2022,10,0.00


In [18]:
data_prs = data_pre.rename(columns={'index': 'date'})
data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
data_prs_info = data_prs.groupby(['零件号','仓库代码','需求流','year', 'month'])['需求数量'].sum().reset_index()
data_prs_info.tail(15)

,零件号,仓库代码,需求流,year,month,需求数量
31,KIT998648,1000-1,MANUAL,2022,9,0
32,KIT998648,1000-1,MANUAL,2022,10,0
33,KIT998648,1000-1,MANUAL,2022,11,0
34,KIT998648,1000-1,MANUAL,2022,12,38288
35,KIT998648,1000-1,MANUAL,2023,1,115940
36,KIT998648,1000-1,MANUAL,2023,2,29876
37,KIT998648,1000-1,MANUAL,2023,3,128552
38,KIT998648,1000-1,MANUAL,2023,4,50482
39,KIT998648,1000-1,MANUAL,2023,5,10232
40,KIT998648,1000-1,MANUAL,2023,6,29998


In [15]:
now_time_future, end_time_train

('2023-11-01', '2023-05-01')